In [1]:
!pip install datasets soundfile librosa matplotlib gradio sentencepiece spaces

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-u6w2ehkb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-u6w2ehkb
  Resolved https://github.com/huggingface/transformers.git to commit b0f0c61899019d316db17a493023828aa44db06d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=10034048 sha256=6d46b717e949f7465f24835e2694db0e31fb0d068c41010344e66ddd731d0a7e
  Stored in directory: /tmp/pip-ephem-wheel-cache-d2d1hbhi/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      S

In [3]:
!pip install torch torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [4]:
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
import datasets
from datasets import load_dataset
import soundfile as sf
import gradio as gr
import spaces  # Import the spaces module
from torch.utils.data import DataLoader
import numpy as np

In [5]:
# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
# Load the StackOverflow dataset with the 'train' split
print("Loading StackOverflow dataset with the 'train' split...")
train_dataset = load_dataset("c17hawke/stackoverflow-dataset", split="train")

Loading StackOverflow dataset with the 'train' split...


train.tsv:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/7.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17536 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7464 [00:00<?, ? examples/s]

In [7]:
print(f"Dataset loaded. Number of training examples: {len(train_dataset)}")

Dataset loaded. Number of training examples: 17536


In [8]:
train_dataset

Dataset({
    features: ['pid', 'label', 'text'],
    num_rows: 17536
})

In [9]:
len(train_dataset)

17536

In [10]:
# Calculate the number of rows for half the dataset
half_size = len(train_dataset) // 16
half_size

1096

In [11]:
# Select the first half of the dataset
train_dataset = train_dataset.select(range(half_size))

print(train_dataset)

Dataset({
    features: ['pid', 'label', 'text'],
    num_rows: 1096
})


In [12]:
# Load the SpeechT5 processor and TTS model
model_name = "microsoft/speecht5_tts"  # SpeechT5 model for TTS
processor = SpeechT5Processor.from_pretrained(model_name)
model = SpeechT5ForTextToSpeech.from_pretrained(model_name).to(device)  # Send model to GPU

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

In [13]:
# Load speaker embeddings from the CMU Arctic xvectors dataset
print("Loading speaker embeddings...")
speaker_embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(speaker_embeddings_dataset[0]['xvector']).unsqueeze(0).to(device)  # Send to GPU

Loading speaker embeddings...


README.md:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

cmu-arctic-xvectors.py:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/585M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/21.3M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

In [14]:
# Preprocess the dataset (use 'text' column)
def preprocess_function(examples):
    inputs = examples['text']  # Use the 'text' column for sentences
    return {'text': inputs}  # Return as 'text' for processing

In [15]:
# Apply preprocessing to the train dataset
#print("Preprocessing the training dataset...")
#train_dataset = dataset['train'].map(preprocess_function, batched=True)
#print("Preprocessing complete. Number of training examples:", len(train_dataset))

# Apply preprocessing to the train dataset
print("Preprocessing the training dataset...")
train_dataset = train_dataset.map(preprocess_function, batched=True)
print(f"Preprocessing complete. Number of training examples: {len(train_dataset)}")

Preprocessing the training dataset...


Map:   0%|          | 0/1096 [00:00<?, ? examples/s]

Preprocessing complete. Number of training examples: 1096


In [16]:
# Create DataLoader for batching
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [28]:
# Fine-tune the SpeechT5 model (TTS)
def train_model():
    print("Starting fine-tuning...")
    scaler = torch.amp.GradScaler(device='cuda')  # Create a gradient scaler for mixed precision
    for epoch in range(5):  # Adjust epochs as needed
        print(f"Epoch {epoch + 1} of 5...")
        for batch_idx, batch in enumerate(train_dataloader):  # Iterate over batches with index
            # Tokenize input text and pad/truncate to a dynamic length based on positional embeddings
            inputs = processor(text=batch['text'], return_tensors="pt", padding=True, truncation=True).to(device)

            # Get the maximum sequence length from positional embeddings
            max_length = model.speecht5.decoder.prenet.encode_positions.pe.shape[1]

            # Truncate input_ids and attention_mask to the maximum length
            inputs['input_ids'] = inputs['input_ids'][:, :max_length]
            inputs['attention_mask'] = inputs['attention_mask'][:, :max_length]

            with torch.amp.autocast(device_type='cuda'):  # Enable autocasting
                # Generate audio output with speaker embeddings
                try:
                    audio = model.generate(inputs['input_ids'], # Removed max_length argument
                                           speaker_embeddings=speaker_embeddings,
                                           attention_mask=inputs['attention_mask'])
                    # Print batch completion message
                    print(f"Batch {batch_idx + 1} completed.")
                except RuntimeError as e:
                    if "The size of tensor a" in str(e):
                        print(f"Error: Input sequence length exceeds maximum supported length ({max_length}). Skipping this batch.")
                        # You can choose to further truncate the input or skip the batch
                        continue
                    else:
                        raise e # Raise other exceptions

            # Cast to float32 before saving
            # audio_float32 = audio.cpu().numpy().astype('float32')
            # Save generated audio for each batch in the epoch
            # sf.write(f'output_epoch{epoch}.wav', audio_float32, 24000)

    print("Fine-tuning completed!")

In [29]:
# Call the training function
train_model()

Starting fine-tuning...
Epoch 1 of 5...
Batch 1 completed.
Error: Input sequence length exceeds maximum supported length (1876). Skipping this batch.
Batch 3 completed.
Batch 4 completed.
Batch 5 completed.
Batch 6 completed.
Batch 7 completed.
Batch 8 completed.
Batch 9 completed.
Batch 10 completed.
Batch 11 completed.
Batch 12 completed.
Batch 13 completed.
Batch 14 completed.
Batch 15 completed.
Batch 16 completed.
Batch 17 completed.
Batch 18 completed.
Batch 19 completed.
Batch 20 completed.
Batch 21 completed.
Batch 22 completed.
Batch 23 completed.
Batch 24 completed.
Batch 25 completed.
Batch 26 completed.
Batch 27 completed.
Batch 28 completed.
Batch 29 completed.
Batch 30 completed.
Batch 31 completed.
Batch 32 completed.
Batch 33 completed.
Batch 34 completed.
Batch 35 completed.
Batch 36 completed.
Batch 37 completed.
Batch 38 completed.
Batch 39 completed.
Batch 40 completed.
Batch 41 completed.
Batch 42 completed.
Batch 43 completed.
Batch 44 completed.
Batch 45 complete

In [ ]:
import pickle
import zipfile
from google.colab import files

# Save the model
model_save_path = '/content/model_filename.pkl'
with open(model_save_path, 'wb') as file:
    pickle.dump(train_model, file)

# Zip the saved model
zip_path = '/content/model_filename.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
    zipf.write(model_save_path)

# Download the zipped model to your local system
files.download(zip_path)